In [ ]:
# dependencias


In [1]:
# paso 01: Eliminar columnas innecesarias
import pandas as pd
import os

# ruta origen
origen = 'D:/academico/8 ciclo/taller web/datos_proyecto/v2'

# Crear carpeta de salida si no existe
os.makedirs(origen + '/transformado', exist_ok=True)

# leer los archivos
alumno = pd.read_csv(origen + '/Alumno.csv')
alumnoCurso = pd.read_csv(origen + '/AlumnoCurso.csv')
alumnoGrupo = pd.read_csv(origen + '/AlumnoGrupo.csv')
alumnoNotas = pd.read_csv(origen + '/AlumnoNotas.csv')
carrera = pd.read_csv(origen + '/Carrera.csv')
competencia = pd.read_csv(origen + '/Competencia.csv')
componenteCompetencia = pd.read_csv(origen + '/ComponenteCompetencia.csv')
curso = pd.read_csv(origen + '/Curso.csv')
cursoCompetencia = pd.read_csv(origen + '/CursoCompetencia.csv')
cursoComponente = pd.read_csv(origen + '/CursoComponente.csv')
periodoAcademico = pd.read_csv(origen + '/PeriodoAcademico.csv')
planEstudios = pd.read_csv(origen + '/PlanEstudios.csv')

# eliminar columnas no deseadas
alumno = alumno.drop(columns=['estado', 'institucionid', 'departamentoid', 'usuarioid'])
alumnoCurso = alumnoCurso.drop(columns=['institucionid', 'departamentoid', 'estado'])
alumnoGrupo = alumnoGrupo.drop(columns=[])
alumnoNotas = alumnoNotas.drop(columns=[])
carrera = carrera.drop(columns=['estado'])
competencia = competencia.drop(columns=['planid', 'institucionid', 'departamentoid'])
componenteCompetencia = componenteCompetencia.drop(columns=['peso'])
curso = curso.drop(columns=['numHorasTeoria', 'numHorasPractica', 'numHorasLaboratorio', 'numHorasCampo', 
                            'numCreditos', 'estado', 'institucionid', 'departamentoid', 'sumilla', 'modalidad'])
cursoCompetencia = cursoCompetencia.drop(columns=[])
cursoComponente = cursoComponente.drop(columns=['evaluacionid', 'peso', 'estado', 'orden', 'padreid', 'nivel', 
                                    'institucionid', 'calculado', 'departamentoid', 'formulaid'])
periodoAcademico = periodoAcademico.drop(columns=['fechaInicio', 'fechaFin', 'modalidad', 'institucionid', 'estado', 'activo', 'departamentoid'])
planEstudios = planEstudios.drop(columns=['institucionid', 'vigencia', 'departamentoid', 'estado'])

# guardar el dataframe en un nuevo archivo
alumno.to_csv(origen + '/transformado/Alumno.csv', index=False)
alumnoCurso.to_csv(origen + '/transformado/AlumnoCurso.csv', index=False)
alumnoGrupo.to_csv(origen + '/transformado/AlumnoGrupo.csv', index=False)
alumnoNotas.to_csv(origen + '/transformado/AlumnoNotas.csv', index=False)
carrera.to_csv(origen + '/transformado/Carrera.csv', index=False)
competencia.to_csv(origen + '/transformado/Competencia.csv', index=False)
componenteCompetencia.to_csv(origen + '/transformado/ComponenteCompetencia.csv', index=False)
curso.to_csv(origen + '/transformado/Curso.csv', index=False)
cursoCompetencia.to_csv(origen + '/transformado/CursoCompetencia.csv', index=False)
cursoComponente.to_csv(origen + '/transformado/CursoComponente.csv', index=False)
periodoAcademico.to_csv(origen + '/transformado/PeriodoAcademico.csv', index=False)
planEstudios.to_csv(origen + '/transformado/PlanEstudios.csv', index=False)

print("Los archivos han sido modificados exitosamente")


Los archivos han sido modificados exitosamente


In [2]:
# paso 02: cargar las colecciones a mongoDB
import pandas as pd
from pymongo import MongoClient

# Conectar a MongoDB
try:
    client = MongoClient('mongodb://localhost:27017/')
    db = client['data_warehouse']
except Exception as e:
    print(f"Error al conectar a MongoDB: {e}")
    exit(1)

origen = 'D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado'

# Función para cargar y actualizar un archivo CSV en una colección de MongoDB
def cargar_y_actualizar_csv_a_mongodb(archivo_csv, nombre_coleccion, id_campos):
    try:
        df = pd.read_csv(archivo_csv)
    except Exception as e:
        print(f"Error al leer el archivo CSV {archivo_csv}: {e}")
        return

    data = df.to_dict(orient='records')
    
    # Actualizar o insertar cada registro en la colección
    for registro in data:
        filtro = {id_campo: registro[id_campo] for id_campo in id_campos}
        db[nombre_coleccion].replace_one(filtro, registro, upsert=True)

# Archivos CSV, nombres de colecciones y columnas identificadoras
archivos_csv = [
    (origen + '/Alumno.csv', 'alumnos', ['id']),
    (origen + '/AlumnoCurso.csv', 'alumno_curso', ['id']),
    (origen + '/AlumnoGrupo.csv', 'alumno_grupo', ['alumnocursoid', 'grupoid']),
    (origen + '/AlumnoNotas.csv', 'alumno_notas', ['alumnonotaid']),
    (origen + '/Carrera.csv', 'carreras', ['id']),
    (origen + '/Competencia.csv', 'competencias', ['id']),
    (origen + '/ComponenteCompetencia.csv', 'componente_competencia', ['id']),
    (origen + '/Curso.csv', 'cursos', ['id']),
    (origen + '/CursoCompetencia.csv', 'curso_competencia', ['id']),
    (origen + '/CursoComponente.csv', 'curso_componente', ['id']),
    (origen + '/PeriodoAcademico.csv', 'periodo_academico', ['id']),
    (origen + '/PlanEstudios.csv', 'plan_estudios', ['id'])
]

# Cargar y actualizar los archivos CSV en MongoDB
for archivo_csv, nombre_coleccion, id_campos in archivos_csv:
    print(f"Procesando {archivo_csv}...")
    cargar_y_actualizar_csv_a_mongodb(archivo_csv, nombre_coleccion, id_campos)

print("Datos cargados y actualizados en MongoDB.")


Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/Alumno.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/AlumnoCurso.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/AlumnoGrupo.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/AlumnoNotas.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/Carrera.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/Competencia.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/ComponenteCompetencia.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/Curso.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/CursoCompetencia.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/transformado/CursoComponente.csv...
Procesando D:/academico/8 ciclo/taller web/datos_proyecto/v2/trans

In [3]:
# Paso 03: establecer las relaciones entre las colecciones
from pymongo import MongoClient

# Conectar a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['data_warehouse']

# Funciones Específicas para Crear Relaciones

def establecer_relacion_alumno_curso():
    for registro in db.alumno_curso.find():
        alumno = db.alumnos.find_one({"id": registro["alumnoid"]})
        curso = db.cursos.find_one({"id": registro["cursoid"]})
        periodo = db.periodo_academico.find_one({"id": registro["periodoid"]})

        if alumno and curso and periodo:
            db.alumno_curso.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "alumno_ref": alumno["_id"],
                    "curso_ref": curso["_id"],
                    "periodo_ref": periodo["_id"]
                }}
            )

def establecer_relacion_alumno_grupo():
    for registro in db.alumno_grupo.find():
        alumnocurso = db.alumno_curso.find_one({"id": registro["alumnocursoid"]})
        if alumnocurso:
            db.alumno_grupo.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "alumnocurso_ref": alumnocurso["_id"]
                }}
            )

def establecer_relacion_alumno_notas():
    for registro in db.alumno_notas.find():
        alumno = db.alumnos.find_one({"id": registro["alumnoid"]})
        curso = db.cursos.find_one({"id": registro["cursoid"]})
        componente = db.curso_componente.find_one({"id": registro["componentenotaid"]})

        if alumno and curso and componente:
            db.alumno_notas.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "alumno_ref": alumno["_id"],
                    "curso_ref": curso["_id"],
                    "componente_ref": componente["_id"]
                }}
            )

def establecer_relacion_componente_competencia():
    for registro in db.componente_competencia.find():
        cursocompetencia = db.curso_competencia.find_one({"id": registro["cursocompetenciaid"]})
        cursocomponente = db.curso_componente.find_one({"id": registro["cursocomponenteid"]})

        if cursocompetencia and cursocomponente:
            db.componente_competencia.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "cursocompetencia_ref": cursocompetencia["_id"],
                    "cursocomponente_ref": cursocomponente["_id"]
                }}
            )

def establecer_relacion_cursos():
    for registro in db.cursos.find():
        planestudios = db.plan_estudios.find_one({"id": registro["planestudiosid"]})
        periodo = db.periodo_academico.find_one({"id": registro["periodoacademicoid"]})
        if planestudios:
            db.cursos.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "planestudios_ref": planestudios["_id"]
                }}
            )
        if periodo:
            db.cursos.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "periodo_ref": periodo["_id"]
                }}
            )

def establecer_relacion_curso_competencia():
    for registro in db.curso_competencia.find():
        curso = db.cursos.find_one({"id": registro["cursoid"]})
        competencia = db.competencias.find_one({"id": registro["competenciaid"]})

        if curso and competencia:
            db.curso_competencia.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "curso_ref": curso["_id"],
                    "competencia_ref": competencia["_id"]
                }}
            )

def establecer_relacion_plan_estudios():
    for registro in db.plan_estudios.find():
        carrera = db.carreras.find_one({"id": registro["carreraid"]})
        if carrera:
            db.plan_estudios.update_one(
                {"_id": registro["_id"]},
                {"$set": {
                    "carrera_ref": carrera["_id"]
                }}
            )

# Ejecutar las funciones para establecer todas las relaciones
establecer_relacion_alumno_curso()
establecer_relacion_alumno_grupo()
establecer_relacion_alumno_notas()
establecer_relacion_componente_competencia()
establecer_relacion_cursos()
establecer_relacion_curso_competencia()
establecer_relacion_plan_estudios()

print("Relaciones establecidas en MongoDB.")


Relaciones establecidas en MongoDB.


In [5]:
# Paso 04: crear el datawarehouse
from pymongo import MongoClient

# Conectar a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['data_warehouse']

# Función para crear el datawarehouse
def crear_datawarehouse():
    # Verificar si la colección datawarehouse existe, si no, crearla
    if 'datawarehouse' in db.list_collection_names():
        db.datawarehouse.drop()

    # Recorrer todos los registros de AlumnoNotas
    for nota in db.alumno_notas.find():
        alumno = db.alumnos.find_one({"_id": nota["alumno_ref"]})
        curso = db.cursos.find_one({"_id": nota["curso_ref"]})
        componente = db.curso_componente.find_one({"_id": nota["componente_ref"]})

        if alumno and curso and componente:
            # Encontrar el AlumnoCurso correspondiente
            alumnocurso = db.alumno_curso.find_one({"alumno_ref": alumno["_id"], "curso_ref": curso["_id"]})
            if alumnocurso:
                periodo = db.periodo_academico.find_one({"_id": alumnocurso["periodo_ref"]})
                if periodo:
                    # Encontrar las competencias del curso relacionadas con el componente específico
                    componente_competencias = db.componente_competencia.find({"cursocomponenteid": componente["id"]})
                    for componente_competencia in componente_competencias:
                        curso_competencia = db.curso_competencia.find_one({"id": componente_competencia["cursocompetenciaid"]})
                        if curso_competencia:
                            competencia = db.competencias.find_one({"id": curso_competencia["competenciaid"]})
                            if competencia:
                                data_warehouse_record = {
                                    "PeriodoAcademico_id": periodo["id"],
                                    "Alumno_id": alumno["id"],
                                    "Curso_id": curso["id"],
                                    "CursoComponente_id": componente["id"],
                                    "Competencia_id": competencia["id"],
                                    "AlumnoNotas_nota": nota["nota"]
                                }
                                db.datawarehouse.insert_one(data_warehouse_record)

crear_datawarehouse()

print("Data warehouse creado en MongoDB.")


Data warehouse creado en MongoDB.
